<a href="https://colab.research.google.com/github/m1shik0/ChromeOS/blob/master/notebooks/official/prediction/pytorch_image_classification_with_prebuilt_serving_containers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Serving PyTorch image models with prebuilt containers on Vertex AI


<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/prediction/pytorch_image_classification_with_prebuilt_serving_containers.ipynb">
      <img src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fprediction%2Fpytorch_image_classification_with_prebuilt_serving_containers.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/prediction/pytorch_image_classification_with_prebuilt_serving_containers.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/prediction/pytorch_image_classification_with_prebuilt_serving_containers.ipynb">
      <img width="32px" src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>


**_NOTE_**: This notebook has been tested in the following environment:

* Python version = 3.9

## Overview

This tutorial demonstrates how to upload and deploy a PyTorch image model using a prebuilt serving container and how to make online and batch predictions.

Vertex AI provides prebuilt containers for serving predictions and explanations from trained model artifacts. Using a pre-built container is generally simpler than creating your own custom container for prediction.

Learn more about [Pre-built containers for prediction](https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers).

### Objective

In this tutorial, you learn how to package and deploy a PyTorch image classification model using a prebuilt Vertex AI container with TorchServe for serving online and batch predictions.

This tutorial uses the following Google Cloud ML services and resources:

- `Vertex AI Model Registry`
- `Vertex AI Model` resources
- `Vertex AI Endpoint` resources

The steps performed include:

- Download a pretrained image model from PyTorch
- Create a custom model handler
- Package model artifacts in a model archive file
- Upload model for deployment
- Deploy model for prediction
- Make online predictions
- Make batch predictions

### Model

This tutorial uses a pretrained image model [resnet18](https://pytorch.org/vision/master/models/generated/torchvision.models.resnet18.html) from the PyTorch TorchVision.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing),
and [Cloud Storage pricing](https://cloud.google.com/storage/pricing),
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Get started

### Install Vertex AI SDK for Python and other required packages


In [2]:
! pip3 install --upgrade --quiet google-cloud-aiplatform==1.23.0 \
                                 tensorflow==2.12.0 \
                                 torch==2.0.0 \
                                 torchvision==0.15.1 \
                                 torch-model-archiver==0.7.1 \
                                 packaging==14.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.0/586.0 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 110.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16

### Restart runtime (Colab only)

To use the newly installed packages, you must restart the runtime on Google Colab.

In [3]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

Authenticate your environment on Google Colab.


In [4]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK for Python

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com). Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

#### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

**If your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

#### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project.

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

### Import libraries

In [ ]:
import base64
import json
import os
import pathlib
import urllib.request

import tensorflow as tf
import torch
from PIL import Image
from torchvision import models

## Download a pre-trained image model

Download the pretrained image model [resnet18](https://pytorch.org/vision/master/models/generated/torchvision.models.resnet18.html) from the PyTorch TorchVision.

In [ ]:
# Create a local directory for model artifacts
model_path = "model"

!rm -r $model_path
!mkdir $model_path

model_name = "resnet-18-custom-handler"
model_file = f"{model_path}/{model_name}.pt"

In [ ]:
# Use scripted mode to save the PyTorch model locally
model = models.resnet18(pretrained=True)
script_module = torch.jit.script(model)
script_module.save(model_file)

## Create a custom model handler

A custom model handler is a Python script that you package with the model when you use the model archiver. The script typically defines how to pre-process input data, invoke the model and post-process the output.

TorchServe has [default handlers](https://pytorch.org/serve/default_handlers.html) for `image_classifier`, `image_segmenter`, `object_detector` and `text_classifier`. In this tutorial, you create a custom handler extending the default [`image_classifier`](https://github.com/pytorch/serve/blob/master/ts/torch_handler/image_classifier.py) handler.


In [ ]:
hander_file = f"{model_path}/custom_handler.py"

In [ ]:
%%writefile {hander_file}

import torch
import torch.nn.functional as F
from ts.torch_handler.image_classifier import ImageClassifier
from ts.utils.util import map_class_to_label


class CustomImageClassifier(ImageClassifier):

    # Only return the top 3 predictions
    topk = 3

    def postprocess(self, data):
        ps = F.softmax(data, dim=1)
        probs, classes = torch.topk(ps, self.topk, dim=1)
        probs = probs.tolist()
        classes = classes.tolist()
        return map_class_to_label(probs, self.mapping, classes)

## Download an index_to_name.json file

PyTorch `image_classifier`, `text_classifier` and `object_detector` can all automatically map from numeric classes (0,1,2...) to friendly strings. To do this, simply include `index_to_name.json` that contains a mapping of class number to friendly name in your model archive file.

In [ ]:
index_to_name_file = f"{model_path}/index_to_name.json"

urllib.request.urlretrieve(
    "https://github.com/pytorch/serve/raw/master/examples/image_classifier/index_to_name.json",
    index_to_name_file,
)

## Package the model artifacts in a model archive file

You package all the model artifacts in a model archive file using the [`Torch model archiver`](https://github.com/pytorch/serve/tree/master/model-archiver).

Note that the prebuilt PyTorch serving containers require the model archive file named as `model.mar` so you need to set the model-name as `model` in the `torch-model-archiver` command.

In [ ]:
# Add torch-model-archiver to the PATH
os.environ["PATH"] = f'{os.environ.get("PATH")}:~/.local/bin'

In [ ]:
!torch-model-archiver -f \
  --model-name model \
  --version 1.0  \
  --serialized-file $model_file \
  --handler $hander_file \
  --extra-files $index_to_name_file \
  --export-path $model_path

## Copy the model artifacts to Cloud Storage

Next, use `gsutil` to copy the model artifacts to your Cloud Storage bucket.

In [ ]:
MODEL_URI = f"{BUCKET_URI}/{model_name}"

!gsutil rm -r $MODEL_URI
!gsutil cp -r $model_path $MODEL_URI
!gsutil ls -al $MODEL_URI

## Upload model for deployment

Next, you upload the [model](https://cloud.google.com/vertex-ai/docs/reference/rest/v1/projects.locations.models) to `Vertex AI Model Registry`, which will create a `Vertex AI Model` resource for your model. This tutorial uses the PyTorch v1.11 container, but for your own use case, you can choose from the list of [PyTorch prebuilt containers](https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers#pytorch).

In [ ]:
DEPLOY_IMAGE_URI = "us-docker.pkg.dev/vertex-ai/prediction/pytorch-cpu.1-11:latest"

deployed_model = aiplatform.Model.upload(
    display_name=model_name,
    serving_container_image_uri=DEPLOY_IMAGE_URI,
    artifact_uri=MODEL_URI,
)

## Deploy model for prediction

Next, deploy your model for online prediction. You set the variable `DEPLOY_COMPUTE` to configure the machine type for the [compute resources](https://cloud.google.com/vertex-ai/docs/predictions/configure-compute) you will use for prediction.

In [ ]:
DEPLOY_COMPUTE = "n1-standard-4"

endpoint = deployed_model.deploy(
    deployed_model_display_name=model_name,
    machine_type=DEPLOY_COMPUTE,
    accelerator_type=None,
    accelerator_count=0,
)

## Make online predictions

### Download an image dataset
In this example, you use the TensorFlow flowers dataset for the input for both online and batch predictions.

In [ ]:
data_dir = tf.keras.utils.get_file(
    "flower_photos",
    origin="https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz",
    untar=True,
)

data_dir = pathlib.Path(data_dir)
images_files = list(data_dir.glob("daisy/*"))

### Get online predictions

You send a `predict` request with encoded input image data to the `endpoint` and get prediction.

In [ ]:
with open(images_files[0], "rb") as f:
    data = {"data": base64.b64encode(f.read()).decode("utf-8")}

response = endpoint.predict(instances=[data])

In [ ]:
prediction = response.predictions[0]
prediction = dict(sorted(prediction.items(), key=lambda item: item[1], reverse=True))

print(prediction)
image = Image.open(images_files[0])
image

## Make batch predictions

### Create the batch input file

You create a batch input file in JSONL format and store the input file in your Cloud Storage bucket.

Learn more about [Input data requirements](https://cloud.google.com/vertex-ai/docs/predictions/get-predictions#input_data_requirements).

In [ ]:
TEST_IMAGE_SIZE = 2
test_image_list = []
for i in range(TEST_IMAGE_SIZE):
    test_image_list.append(str(images_files[i]))

gcs_input_uri = f"{BUCKET_URI}/test_images.json"

with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    for test_image in test_image_list:
        with open(test_image, "rb") as image_f:
            data = {"data": base64.b64encode(image_f.read()).decode("utf-8")}
            f.write(json.dumps(data) + "\n")

### Submit a batch prediction job

In [ ]:
JOB_DISPLAY_NAME = f"{model_name}_batch_predict_job_unique"

batch_predict_job = deployed_model.batch_predict(
    job_display_name=JOB_DISPLAY_NAME,
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_URI,
    instances_format="jsonl",
    model_parameters=None,
    machine_type=DEPLOY_COMPUTE,
)

### Get batch predictions

After the batch job completes, the results are written to the Cloud Storage output bucket you specified in the batch request. You call the method `iter_outputs()` to get a list of each Cloud Storage file generated with the results.

In [ ]:
bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_files = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction.results"):
        prediction_files.append(blob.name)

In [ ]:
prediction_file = prediction_files[0]

results = []
gfile_name = f"{BUCKET_URI}/{prediction_file}"
with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
    for line in gfile.readlines():
        results.append(json.loads(line))

# Take one result as an example and print out the prediction.
prediction = results[0]["prediction"]
prediction = dict(sorted(prediction.items(), key=lambda item: item[1], reverse=True))
print(prediction)

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial.

In [ ]:
endpoint.undeploy_all()
endpoint.delete()

deployed_model.delete()
batch_predict_job.delete()

delete_bucket = False
if delete_bucket:
    ! gsutil -m rm -r $BUCKET_URI